In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.models as models
from torchvision.models import MobileNet_V2_Weights
from PreprocessingFunctions import *
from ModelEval import *

In [2]:
# data_transform = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                         std=[0.229, 0.224, 0.225])])
# train_set = ImageFolder("data/small_train_172", transform=data_transform)
# train_loader = DataLoader(train_set, batch_size=16, shuffle=True, num_workers=1)
# val_set = ImageFolder("data/small_validation_172", transform=data_transform)
# val_loader = DataLoader(val_set, batch_size=1, shuffle=False, num_workers=1)

# class_stats = compute_class_stats(train_loader)

In [4]:
class_stats = (172, 37389, [269, 63, 500, 290, 111, 238, 500, 111, 324, 111, 46, 500, 38, 83, 46, 396, 246, 500,
                            167, 51, 92, 54, 180, 97, 195, 60, 222, 81, 41, 277, 315, 348, 86, 219, 67, 45, 209,
                            61, 135, 195, 44, 500, 351, 500, 453, 500, 500, 332, 332, 68, 189, 83, 77, 396, 242,
                            255, 50, 326, 500, 116, 500, 500, 234, 34, 38, 114, 278, 90, 500, 187, 47, 208, 58,
                            500, 279, 500, 172, 207, 500, 99, 207, 63, 500, 55, 113, 432, 339, 34, 168, 36, 86,
                            50, 257, 500, 179, 209, 98, 42, 87, 65, 41, 83, 112, 261, 500, 394, 142, 147, 156,
                            84, 500, 256, 71, 59, 270, 264, 467, 73, 153, 58, 132, 201, 131, 69, 74, 43, 230,
                            108, 500, 75, 218, 343, 58, 167, 239, 216, 99, 500, 202, 141, 72, 87, 500, 140, 70,
                            72, 463, 91, 138, 213, 126, 80, 97, 500, 280, 42, 53, 500, 51, 500, 93, 145, 264,
                            500, 495, 281, 500, 87, 307, 201, 156, 500])

In [11]:
model = models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)
#model = models.mobilenet_v2()

model.classifier[1] = nn.Linear(1280, 172)
#model.load_state_dict(torch.load("saved_models/ft_mobile_5.pt"))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [12]:
def finetune(model, train_stats, epochs=10, batch_size=16, lr=0.001, L2_reg=0.01, 
             train_path="data/small_train_sifted", val_path="data/small_validation_sifted", 
             saved_filepath="saved_models/fine-tuned_model.pt"):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Device in use: {device}")
    if device == "cuda":
        torch.cuda.empty_cache()
        #torch.cuda.max_memory_allocated(max_split_size_mb=1024)
    model.to(device)
    print("==> Starting Data Preparation...")
    data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
    ])
    train_set = ImageFolder(train_path, transform=data_transform)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=1)
    val_set = ImageFolder(val_path, transform=data_transform)
    val_loader = DataLoader(val_set, batch_size=1, shuffle=False, num_workers=1)
    
    num_classes, total_samples, class_samples = train_stats
    weights = torch.FloatTensor([total_samples / (num_classes * s) for s in class_samples]).to(device)
    loss_function = nn.CrossEntropyLoss(weight=weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=L2_reg)
    
    best_acc = 0
    
    print("==> Training Initiated...")
    for epoch in range(0, epochs):
        print(f"Epoch {epoch}/{epochs}")
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model.forward(inputs)
            loss = loss_function(outputs, targets)
            loss.backward()
            
            optimizer.step()
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
        print(f"Train Loss: {round(train_loss,3)} | Train Acc: {round(correct/total,3)}")
        #scheduler.step()
        
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model.forward(inputs)
                loss = loss_function(outputs, targets)
                
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
        val_acc = correct/total
        
        print(f"Val Loss:   {round(val_loss,3)} | Val Acc:   {round(val_acc,3)}")
        
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving Model...")
            torch.save(model.state_dict(), saved_filepath)
    
    print("=========> Training Complete <=========")
    print(f"Best Validation Accuracy: {best_acc}")


In [ ]:
finetune(model, class_stats, epochs=100, batch_size=8, lr=0.0003, L2_reg=0.001,
         train_path="data/small_train_172", val_path = "data/small_validation_172",
         saved_filepath="saved_models/ft_mobile172_1.pt")

Device in use: cuda
==> Starting Data Preparation...
==> Training Initiated...
Epoch 0/100
Train Loss: 17107.441 | Train Acc: 0.249
Val Loss:   30139.9 | Val Acc:   0.004
Saving Model...
Epoch 1/100
Train Loss: 10983.393 | Train Acc: 0.471
Val Loss:   34688.599 | Val Acc:   0.005
Saving Model...
Epoch 2/100
Train Loss: 9255.029 | Train Acc: 0.547
Val Loss:   34908.639 | Val Acc:   0.006
Saving Model...
Epoch 3/100


In [2]:
data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])])
val_set = ImageFolder("data/small_validation_sifted", transform=data_transform)
val_loader = DataLoader(val_set, batch_size=1, shuffle=False, num_workers=1)

model = models.mobilenet_v2()
model.classifier[1] = nn.Linear(1280, 132)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.load_state_dict(torch.load("saved_models/ft_mobile_6.pt"))

#generate_confusion_matrix(model, val_loader, "data/small_train_sifted")
test_model(model, val_loader, top3top5=True)

Model accuracy: 3892/4824 ===> 80.67993366500829%
Top 3 accuracy: 4317/4824 ===> 89.49004975124379%
Top 5 accuracy: 4440/4824 ===> 92.03980099502488%
Top 10 accuracy: 4566/4824 ==> 94.65174129353234%
